In [16]:
import random
import pandas as pd
import datetime

# Paso 1: Generar Usuarios
def generar_usuarios(num_usuarios):
    users = []
    for user_id in range(1, num_usuarios + 1):
        username = f"user_{user_id}"
        password = f"pass_{user_id}"
        users.append([user_id, username, password])
    return pd.DataFrame(users, columns=['user_id', 'username', 'password'])

# Función para clasificar usuarios en activos y menos activos
def clasificar_usuarios(df_users):
    usuarios_activos = df_users.sample(frac=0.2).user_id.tolist()  # 20% de usuarios activos
    usuarios_menos_activos = df_users[~df_users['user_id'].isin(usuarios_activos)].user_id.tolist()  # 80% menos activos
    return usuarios_activos, usuarios_menos_activos

# Función para generar interacciones basadas en usuarios activos y menos activos
def generar_interacciones(num_interacciones, df_movies, df_users, usuarios_activos, usuarios_menos_activos, emociones):
    interacciones = []
    historial_interacciones = {}

    # Proporción: por cada 6 interacciones, 5 serán 'shown' y 1 será 'view'
    view_ratio = 1 / 6
    shown_ratio = 5 / 6

    for _ in range(num_interacciones):
        # Seleccionar un usuario activo o menos activo
        if random.random() > 0.3:
            user_id = random.choice(usuarios_activos)
        else:
            user_id = random.choice(usuarios_menos_activos)

        # Seleccionar una emoción aleatoria
        emotion = random.choice(emociones)

        # Filtrar las películas que contengan esa emoción
        peliculas_filtradas = df_movies[df_movies['emotions'].apply(lambda x: emotion in x)]

        # Solo continuar si hay películas que coincidan con la emoción seleccionada
        if not peliculas_filtradas.empty:
            # Seleccionar una película aleatoria de las filtradas
            movie_id = peliculas_filtradas.sample(1).index[0]

            # Inicializar el historial de interacciones para el usuario si no existe
            if user_id not in historial_interacciones:
                historial_interacciones[user_id] = {}

            # Inicializar el historial para la película si no existe
            if movie_id not in historial_interacciones[user_id]:
                historial_interacciones[user_id][movie_id] = []

            # Determinar si esta interacción será "shown" o "view" basándonos en la proporción
            if len(historial_interacciones[user_id][movie_id]) == 0:
                if random.random() < shown_ratio:
                    interaction_type = 'shown'
                else:
                    interaction_type = 'view'
            else:
                continue  # Si ya existe una interacción para esa película, omitimos

            # Generar una fecha aleatoria en el último año
            date = datetime.datetime.now() - datetime.timedelta(days=random.randint(0, 365))

            # Añadir interacción a la lista
            interacciones.append([user_id, movie_id, emotion, interaction_type, date])

            # Registrar la interacción para evitar duplicados
            historial_interacciones[user_id][movie_id].append(interaction_type)

    return pd.DataFrame(interacciones, columns=['user_id', 'movie_id', 'emotion', 'interaction_type', 'date'])

# Función para generar favoritos basados en las interacciones (solo 'view')
def generar_favoritos(df_interactions):
    favoritos = []

    # Solo seleccionamos interacciones de tipo 'view' para ser añadidas a favoritos
    interacciones_vistas = df_interactions[df_interactions['interaction_type'] == 'view']

    # Seleccionar el 30% de las interacciones de tipo 'view'
    num_favoritos = int(len(interacciones_vistas) * 0.3)

    for _, interaccion_seleccionada in interacciones_vistas.sample(num_favoritos).iterrows():
        user_id = interaccion_seleccionada['user_id']
        movie_id = interaccion_seleccionada['movie_id']
        fecha_vista = interaccion_seleccionada['date']
        
        # La fecha en la que se marca como favorita debe ser posterior o igual a la fecha de vista
        dias_despues = random.randint(0, (datetime.datetime.now() - fecha_vista).days)
        date_added = fecha_vista + datetime.timedelta(days=dias_despues)

        favoritos.append([user_id, movie_id, date_added])

    return pd.DataFrame(favoritos, columns=['user_id', 'movie_id', 'date_added'])

# Función para generar ratings basados en el 50% de los favoritos
def generar_ratings(df_favorites):
    ratings = []

    # Seleccionar solo el 50% de las películas que han sido marcadas como favoritas
    num_ratings = int(len(df_favorites) * 0.5)

    for _, favorito_seleccionado in df_favorites.sample(num_ratings).iterrows():
        user_id = favorito_seleccionado['user_id']
        movie_id = favorito_seleccionado['movie_id']
        fecha_favorito = favorito_seleccionado['date_added']
        
        # La fecha de rating debe ser igual o posterior a la fecha en que fue añadido a favoritos
        dias_despues = random.randint(0, (datetime.datetime.now() - fecha_favorito).days)
        date_rated = fecha_favorito + datetime.timedelta(days=dias_despues)
        
        # Asignar un rating entre 1 y 10, con cierto sesgo a ratings más altos
        rating = random.choices(range(1, 11), weights=[1, 1, 2, 2, 3, 3, 4, 4, 5, 5], k=1)[0]
        
        # Añadir el rating
        ratings.append([user_id, movie_id, rating, date_rated])

    return pd.DataFrame(ratings, columns=['user_id', 'movie_id', 'rating', 'date'])

In [29]:
# Cargar el dataset de películas
df_movies = pd.read_csv('imdb_clean.csv')  # Asegúrate de que el archivo csv esté disponible
df_movies.index = df_movies.index + 1  # Asegurar que el índice comienza en 1 para simular movie_id

# Crear DataFrame de usuarios con nombres y contraseñas
num_usuarios = 1000
df_users = generar_usuarios(num_usuarios)

# Definir las emociones disponibles
emociones = ['Happy', 'Sad', 'Excited', 'Relaxed', 'Scared', 'Inspired', 'Sweet']

# Clasificar usuarios en activos y menos activos
usuarios_activos, usuarios_menos_activos = clasificar_usuarios(df_users)

# Generar interacciones
df_interactions = generar_interacciones(50000, df_movies, df_users, usuarios_activos, usuarios_menos_activos, emociones)

# Generar favoritos (30% de las películas vistas)
df_favorites = generar_favoritos(df_interactions)

# Generar ratings (50% de las películas favoritas)
df_ratings = generar_ratings(df_favorites)

# Mostrar datos generados
print(df_users.head())
print(df_interactions.head())
print(df_favorites.head())
print(df_ratings.head())

   user_id username password
0        1   user_1   pass_1
1        2   user_2   pass_2
2        3   user_3   pass_3
3        4   user_4   pass_4
4        5   user_5   pass_5
   user_id  movie_id   emotion interaction_type                       date
0      667      1188    Scared            shown 2024-01-02 16:51:51.089748
1      368      3137    Scared            shown 2024-01-21 16:51:51.099750
2      128      2319   Excited             view 2024-09-08 16:51:51.104751
3      477      1770  Inspired            shown 2024-02-18 16:51:51.114748
4      891      5475  Inspired            shown 2023-10-26 16:51:51.117746
   user_id  movie_id                 date_added
0      826      6451 2024-06-13 16:56:12.353287
1      929      9138 2024-06-01 16:52:15.985547
2      462       535 2024-09-01 16:52:03.023451
3      821      2692 2024-09-30 16:53:10.659004
4      927      8496 2024-06-26 16:53:18.502374
   user_id  movie_id  rating                       date
0       83      6250       7 202

In [30]:
# Guardar los datos generados en CSV
df_users.to_csv('usuarios_generados.csv', index=False)
df_interactions.to_csv('interacciones_generadas.csv', index=False)
df_favorites.to_csv('favoritos_generados.csv', index=False)
df_ratings.to_csv('ratings_generados.csv', index=False)

In [31]:
import mysql.connector

# Conectar a la base de datos MySQL
conn = mysql.connector.connect(
    host="localhost",  
    user="root",  
    password="123456",  
    database="movie_recommendations"
)
c = conn.cursor()

In [32]:
# Inserción en base de datos (si deseas trabajar directamente en MySQL)
def insertar_usuarios(df_users):
    for _, row in df_users.iterrows():
        query = """
        INSERT INTO users (user_id, username, password) 
        VALUES (%s, %s, %s)
        """
        values = (row['user_id'], row['username'], row['password'])
        c.execute(query, values)
    conn.commit()

def insertar_interacciones(df_interactions):
    for _, row in df_interactions.iterrows():
        query = """
        INSERT INTO interactions (user_id, movie_id, emotion, interaction_type, date) 
        VALUES (%s, %s, %s, %s, %s)
        """
        values = (row['user_id'], row['movie_id'], row['emotion'], row['interaction_type'], row['date'])
        c.execute(query, values)
    conn.commit()

def insertar_favoritos(df_favorites):
    for _, row in df_favorites.iterrows():
        query = """
        INSERT INTO favorites (user_id, movie_id, date_added) 
        VALUES (%s, %s, %s)
        """
        values = (row['user_id'], row['movie_id'], row['date_added'])
        c.execute(query, values)
    conn.commit()

def insertar_ratings(df_ratings):
    for _, row in df_ratings.iterrows():
        query = """
        INSERT INTO ratings (user_id, movie_id, rating, date) 
        VALUES (%s, %s, %s, %s)
        """
        values = (row['user_id'], row['movie_id'], row['rating'], row['date'])
        c.execute(query, values)
    conn.commit()

# Llamar a las funciones de inserción
insertar_usuarios(df_users)
insertar_interacciones(df_interactions)
insertar_favoritos(df_favorites)
insertar_ratings(df_ratings)

# Cerrar conexión a la base de datos
conn.close()

In [38]:
user_id = 1

# Filtrar interacciones del usuario
interacciones_usuario = df_interactions[df_interactions['user_id'] == user_id]
print(interacciones_usuario)

# Filtrar favoritos del usuario
favoritos_usuario = df_favorites[df_favorites['user_id'] == user_id]
print(favoritos_usuario)

# Filtrar ratings del usuario
ratings_usuario = df_ratings[df_ratings['user_id'] == user_id]
print(ratings_usuario)


       user_id  movie_id   emotion interaction_type                       date
207          1      3343   Relaxed            shown 2023-12-21 16:51:52.734724
223          1      2133   Relaxed            shown 2023-12-28 16:51:52.906726
505          1      1243     Sweet            shown 2023-12-02 16:51:55.916726
572          1      8161  Inspired             view 2024-08-07 16:51:56.462726
669          1      8617   Excited            shown 2024-04-28 16:51:57.228253
...        ...       ...       ...              ...                        ...
41350        1      3080    Scared            shown 2024-03-01 16:56:37.246179
41411        1      3221   Excited             view 2024-09-28 16:56:37.617226
41660        1      2059     Sweet            shown 2023-12-08 16:56:39.052108
41684        1      6353     Happy            shown 2023-12-21 16:56:39.176442
41968        1      2424     Happy            shown 2024-09-24 16:56:40.887141

[159 rows x 5 columns]
      user_id  movie_id     

In [43]:
df_favorites

,user_id,movie_id,date_added
0,826,6451,2024-06-13 16:56:12.353287
1,929,9138,2024-06-01 16:52:15.985547
2,462,535,2024-09-01 16:52:03.023451
3,821,2692,2024-09-30 16:53:10.659004
4,927,8496,2024-06-26 16:53:18.502374
...,...,...,...
2134,638,5459,2024-10-07 16:55:59.866155
2135,771,2078,2023-12-16 16:52:32.880448
2136,878,2762,2024-05-29 16:51:51.967147
2137,522,3165,2024-07-27 16:56:34.631812
